In [1]:
import numpy as np 
import math
import random as rd
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
def create_data(n_rows, n_cols,choice):
    x_sample=[]
    col=[]
    while len(x_sample)<n_rows:
        if choice:
            col=[rd.randint(-1000,1000) for _ in range(n_cols)]
        else:
            col=[rd.choice([0,1]) for _ in range(n_cols)]
        x_sample.append(col)
    x_sample= np.array(x_sample)
    return x_sample

x = create_data(1000, 42,True)
y= create_data(1000, 1,False)


In [2]:
x_train = x[:700]
x_test=x[700:]
y_train=y[:700]
y_test=y[700:]
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

(700, 42) (700, 1) (300, 42) (300, 1)


In [3]:
# -*- coding: utf-8 -*-
"""Inception-ResNet V2 model for Keras.

Model naming and structure follows TF-slim implementation (which has some additional
layers and different number of filters from the original arXiv paper):
https://github.com/tensorflow/models/blob/master/slim/nets/inception_resnet_v2.py

Pre-trained ImageNet weights are also converted from TF-slim, which can be found in:
https://github.com/tensorflow/models/tree/master/slim#pre-trained-models

# Reference
- [Inception-v4, Inception-ResNet and the Impact of
   Residual Connections on Learning](https://arxiv.org/abs/1602.07261)

"""
from __future__ import print_function
from __future__ import absolute_import

import warnings
import numpy as np
from keras.layers import Input, Conv1D, BatchNormalization, Activation, add, MaxPooling1D, GlobalAveragePooling1D,AveragePooling1D,Concatenate, Dense
from keras.models import Model
import keras.backend as K




def preprocess_input(x):
    """Preprocesses a numpy array encoding a batch of images.

    This function applies the "Inception" preprocessing which converts
    the RGB values from [0, 255] to [-1, 1]. Note that this preprocessing
    function is different from `imagenet_utils.preprocess_input()`.

    # Arguments
        x: a 4D numpy array consists of RGB values within [0, 255].

    # Returns
        Preprocessed array.
    """
    x /= 255.
    x -= 0.5
    x *= 2.
    return x


def conv1d_bn(x,
              filters,
              kernel_size,
              strides=1,
              padding='same',
              activation='relu',
              name=None):
    bn_axis=1
    x = Conv1D(filters,
               kernel_size,
               strides=strides,
               padding=padding,
               name=name)(x)
    bn_name = None if name is None else name + '_bn'
    x = BatchNormalization(axis=bn_axis, scale=False, name=bn_name)(x)
    if activation is not None:
        ac_name = None if name is None else name + '_ac'
        x = Activation(activation, name=ac_name)(x)
    return x

from keras.layers import Lambda, ZeroPadding1D

def adjust_padding(branches):
    max_length = max(K.int_shape(branch)[1] for branch in branches)
    adjusted_branches = []
    for branch in branches:
        branch_length = K.int_shape(branch)[1]
        if branch_length < max_length:
            padding = max_length - branch_length
            branch = ZeroPadding1D(padding=(0, padding))(branch)
        adjusted_branches.append(branch)
    return adjusted_branches

def check_and_adjust_branches(branches, channel_axis):
    max_length = max([K.int_shape(branch)[channel_axis] for branch in branches])
    adjusted_branches = []
    for branch in branches:
        branch_length = K.int_shape(branch)[channel_axis]
        if branch_length != max_length:
            # Adjust the size of the branch
            # This can be done using ZeroPadding1D, Cropping1D, or other methods
            # For example, using ZeroPadding1D:
            padding_size = (max_length - branch_length)
            padding = (padding_size // 2, padding_size - padding_size // 2)
            branch = ZeroPadding1D(padding=padding)(branch)
        adjusted_branches.append(branch)
    return adjusted_branches

def adjust_branches_for_concat(branches):
    adjusted_branches = []
    for branch in branches:
        if K.int_shape(branch)[2] != 96:
            # Thêm đệm để kích thước chiều kênh tăng lên 96
            padding_size = 96 - K.int_shape(branch)[2]
            left_pad = padding_size // 2
            right_pad = padding_size - left_pad
            branch = ZeroPadding1D(padding=(left_pad, right_pad))(branch)
        adjusted_branches.append(branch)
    return adjusted_branches


def inception_resnet_block(x, scale, block_type, block_idx, activation='relu'):
 
    if block_type == 'block35':
        target_length = K.int_shape(x)[1]
        branch_0 = conv1d_bn(x, 32, 1)
        branch_1 = conv1d_bn(x, 32, 1)
        branch_1 = conv1d_bn(branch_1, 32, 3)
        branch_2 = conv1d_bn(x, 32, 1)
        branch_2 = conv1d_bn(branch_2, 48, 3)
        branch_2 = conv1d_bn(branch_2, 64, 3)
        branches = [branch_0, branch_1, branch_2]
    elif block_type == 'block17':
        target_length = K.int_shape(x)[1]
        branch_0 = conv1d_bn(x, 192, 1)
        branch_1 = conv1d_bn(x, 128, 1)
        branch_1 = conv1d_bn(branch_1, 160, 1)
        branch_1 = conv1d_bn(branch_1, 192, 7)
        branches = [branch_0, branch_1]
    elif block_type == 'block8':
        target_length = K.int_shape(x)[1]
        branch_0 = conv1d_bn(x, 192, 1)
        branch_1 = conv1d_bn(x, 192, 1)
        branch_1 = conv1d_bn(branch_1, 224, 1)
        branch_1 = conv1d_bn(branch_1, 256, 3)
        branches = [branch_0, branch_1]
    else:
        raise ValueError('Unknown Inception-ResNet block type. '
                         'Expects "block35", "block17" or "block8", '
                         'but got: ' + str(block_type))

    block_name = block_type + '_' + str(block_idx)
    channel_axis = -1
    branches = check_and_adjust_branches(branches, channel_axis=1)
    mixed = Concatenate(axis=channel_axis, name=block_name + '_mixed')(branches)
    up = conv1d_bn(mixed,
                   K.int_shape(x)[channel_axis],
                   1,
                   activation=None,
                   name=block_name + '_conv')

    x = Lambda(lambda inputs, scale: inputs[0] + inputs[1] * scale,
               output_shape=K.int_shape(x)[1:],
               arguments={'scale': scale},
               name=block_name)([x, up])
    if activation is not None:
        x = Activation(activation, name=block_name + '_ac')(x)
    return x


def InceptionResNetV2():

    # Determine proper input shape
    input_shape =(42,1)
    x_input = Input(shape=input_shape)

    # Stem block: 35 x 35 x 192
    x = conv1d_bn(x_input, 32, 3, strides=2, padding='valid')
    x = conv1d_bn(x, 32, 3, padding='valid')
    x = conv1d_bn(x, 64, 3)
    x = MaxPooling1D(3, strides=2)(x)
    x = conv1d_bn(x, 80, 1, padding='valid')
    x = conv1d_bn(x, 192, 3, padding='valid')
    x = MaxPooling1D(3, strides=2)(x)

    # Mixed 5b (Inception-A block): 35 x 35 x 320
    branch_0 = conv1d_bn(x, 96, 1)
    branch_1 = conv1d_bn(x, 48, 1)
    branch_1 = conv1d_bn(branch_1, 64, 5)
    branch_2 = conv1d_bn(x, 64, 1)
    branch_2 = conv1d_bn(branch_2, 96, 3)
    branch_2 = conv1d_bn(branch_2, 96, 3)
    branch_pool = AveragePooling1D(3, strides=1, padding='same')(x)
    branch_pool = conv1d_bn(branch_pool, 64, 1)
    target_shape = (None, 2, 96)
    branches = [branch_0, branch_1, branch_2, branch_pool]
    channel_axis = -1
    #branches = adjust_branches_for_concat(branches)
    x = Concatenate(axis=-1, name='mixed_5b')(branches)

    # 10x block35 (Inception-ResNet-A block): 35 x 35 x 320
    for block_idx in range(1, 11):
        x = inception_resnet_block(x,
                                   scale=0.17,
                                   block_type='block35',
                                   block_idx=block_idx)

    # Mixed 6a (Reduction-A block): 17 x 17 x 1088
    branch_0 = conv1d_bn(x, 384, 3, strides=2, padding='same')
    branch_1 = conv1d_bn(x, 256, 1)
    branch_1 = conv1d_bn(branch_1, 256, 3)
    branch_1 = conv1d_bn(branch_1, 384, 3, strides=2, padding='same')
    branch_pool = MaxPooling1D(3, strides=2, padding='same')(x)
    branches = [branch_0, branch_1, branch_pool]
    x = Concatenate(axis=channel_axis, name='mixed_6a')(branches)

    # 20x block17 (Inception-ResNet-B block): 17 x 17 x 1088
    for block_idx in range(1, 21):
        x = inception_resnet_block(x,
                                   scale=0.1,
                                   block_type='block17',
                                   block_idx=block_idx)

    # Mixed 7a (Reduction-B block): 8 x 8 x 2080
    branch_0 = conv1d_bn(x, 256, 1)
    branch_0 = conv1d_bn(branch_0, 384, 3, strides=2, padding='same')
    branch_1 = conv1d_bn(x, 256, 1)
    branch_1 = conv1d_bn(branch_1, 288, 3, strides=2, padding='same')
    branch_2 = conv1d_bn(x, 256, 1)
    branch_2 = conv1d_bn(branch_2, 288, 3)
    branch_2 = conv1d_bn(branch_2, 320, 3, strides=2, padding='same')
    branch_pool = MaxPooling1D(3, strides=2, padding='same')(x)
    branches = [branch_0, branch_1, branch_2, branch_pool]
    x = Concatenate(axis=channel_axis, name='mixed_7a')(branches)

    # 10x block8 (Inception-ResNet-C block): 8 x 8 x 2080
    for block_idx in range(1, 10):
        x = inception_resnet_block(x,
                                   scale=0.2,
                                   block_type='block8',
                                   block_idx=block_idx)
    x = inception_resnet_block(x,
                               scale=1.,
                               activation=None,
                               block_type='block8',
                               block_idx=10)

    # Final convolution block: 8 x 8 x 1536
    x = conv1d_bn(x, 1536, 1, name='conv_7b')
    x = GlobalAveragePooling1D(name='avg_pool')(x)
    x = Dense(1, activation='softmax', name='predictions')(x)

 

    # Create model
    model = Model(x_input, x, name='inception_resnet_v2')

    

    return model


model = InceptionResNetV2()
lr=0.0001
model.compile(optimizer=Adam(lr=lr), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


Model: "inception_resnet_v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 42, 1)]      0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 20, 32)       128         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 20, 32)      60          ['conv1d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)        (None, 20, 32)       0           ['batch_normali

 ormalization)                                                                                    
                                                                                                  
 batch_normalization_11 (BatchN  (None, 2, 64)       6           ['conv1d_11[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_5 (Activation)      (None, 2, 96)        0           ['batch_normalization_5[0][0]']  
                                                                                                  
 activation_7 (Activation)      (None, 2, 64)        0           ['batch_normalization_7[0][0]']  
                                                                                                  
 activation_10 (Activation)     (None, 2, 96)        0           ['batch_normalization_10[0][0]'] 
          

 conv1d_22 (Conv1D)             (None, 2, 48)        4656        ['activation_21[0][0]']          
                                                                                                  
 batch_normalization_19 (BatchN  (None, 2, 32)       6           ['conv1d_19[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_22 (BatchN  (None, 2, 48)       6           ['conv1d_22[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_19 (Activation)     (None, 2, 32)        0           ['batch_normalization_19[0][0]'] 
                                                                                                  
 activatio

 activation_24 (Activation)     (None, 2, 32)        0           ['batch_normalization_24[0][0]'] 
                                                                                                  
 activation_26 (Activation)     (None, 2, 32)        0           ['batch_normalization_26[0][0]'] 
                                                                                                  
 activation_29 (Activation)     (None, 2, 64)        0           ['batch_normalization_29[0][0]'] 
                                                                                                  
 block35_3_mixed (Concatenate)  (None, 2, 128)       0           ['activation_24[0][0]',          
                                                                  'activation_26[0][0]',          
                                                                  'activation_29[0][0]']          
                                                                                                  
 block35_3

 conv1d_37 (Conv1D)             (None, 2, 32)        10272       ['block35_4_ac[0][0]']           
                                                                                                  
 conv1d_40 (Conv1D)             (None, 2, 48)        4656        ['activation_39[0][0]']          
                                                                                                  
 batch_normalization_37 (BatchN  (None, 2, 32)       6           ['conv1d_37[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_40 (BatchN  (None, 2, 48)       6           ['conv1d_40[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activatio

...

                                                                                                  
 block8_2_conv_bn (BatchNormali  (None, 1, 2080)     3           ['block8_2_conv[0][0]']          
 zation)                                                                                          
                                                                                                  
 block8_2 (Lambda)              (None, 1, 2080)      0           ['block8_1_ac[0][0]',            
                                                                  'block8_2_conv_bn[0][0]']       
                                                                                                  
 block8_2_ac (Activation)       (None, 1, 2080)      0           ['block8_2[0][0]']               
                                                                                                  
 conv1d_172 (Conv1D)            (None, 1, 192)       399552      ['block8_2_ac[0][0]']            
          

 block8_4_conv_bn (BatchNormali  (None, 1, 2080)     3           ['block8_4_conv[0][0]']          
 zation)                                                                                          
                                                                                                  
 block8_4 (Lambda)              (None, 1, 2080)      0           ['block8_3_ac[0][0]',            
                                                                  'block8_4_conv_bn[0][0]']       
                                                                                                  
 block8_4_ac (Activation)       (None, 1, 2080)      0           ['block8_4[0][0]']               
                                                                                                  
 conv1d_180 (Conv1D)            (None, 1, 192)       399552      ['block8_4_ac[0][0]']            
                                                                                                  
 batch_nor

 zation)                                                                                          
                                                                                                  
 block8_6 (Lambda)              (None, 1, 2080)      0           ['block8_5_ac[0][0]',            
                                                                  'block8_6_conv_bn[0][0]']       
                                                                                                  
 block8_6_ac (Activation)       (None, 1, 2080)      0           ['block8_6[0][0]']               
                                                                                                  
 conv1d_188 (Conv1D)            (None, 1, 192)       399552      ['block8_6_ac[0][0]']            
                                                                                                  
 batch_normalization_188 (Batch  (None, 1, 192)      3           ['conv1d_188[0][0]']             
 Normaliza

                                                                                                  
 block8_8 (Lambda)              (None, 1, 2080)      0           ['block8_7_ac[0][0]',            
                                                                  'block8_8_conv_bn[0][0]']       
                                                                                                  
 block8_8_ac (Activation)       (None, 1, 2080)      0           ['block8_8[0][0]']               
                                                                                                  
 conv1d_196 (Conv1D)            (None, 1, 192)       399552      ['block8_8_ac[0][0]']            
                                                                                                  
 batch_normalization_196 (Batch  (None, 1, 192)      3           ['conv1d_196[0][0]']             
 Normalization)                                                                                   
          

 block8_10 (Lambda)             (None, 1, 2080)      0           ['block8_9_ac[0][0]',            
                                                                  'block8_10_conv_bn[0][0]']      
                                                                                                  
 conv_7b (Conv1D)               (None, 1, 1536)      3196416     ['block8_10[0][0]']              
                                                                                                  
 conv_7b_bn (BatchNormalization  (None, 1, 1536)     3           ['conv_7b[0][0]']                
 )                                                                                                
                                                                                                  
 conv_7b_ac (Activation)        (None, 1, 1536)      0           ['conv_7b_bn[0][0]']             
                                                                                                  
 avg_pool 

In [4]:
earlystopping = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 30)

early_stopping = EarlyStopping(monitor='val_accuracy',
    min_delta=0.00005,
    patience=11,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=30,
    min_lr=0.000001,
    verbose=1,
)
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import confusion_matrix

class ConfusionMatrixCallback(Callback):
    def __init__(self, validation_data):
        self.validation_data = validation_data
        self.relist={}
        self.acc=[]

    def on_epoch_end(self, epoch, logs=None):
        X_val, y_val = self.validation_data
        y_pred = self.model.predict(X_val)
        y_pred_binary = (y_pred > 0.5).astype(int)

        cm = confusion_matrix(y_val, y_pred_binary)
        print(f"Confusion Matrix after Epoch {epoch + 1}:\n{cm}")
        print(classification_report(y_test,y_pred_binary))
        acc = float(classification_report(y_test,y_pred_binary).split('\n')[3].split()[2])
        self.acc.append(acc)
        self.relist[acc]=y_pred_binary

# ... Định nghĩa mô hình ...

# Tạo đối tượng callback
confusion_matrix_callback = ConfusionMatrixCallback(validation_data=(x_test,y_test))

callbacks = [early_stopping,lr_scheduler]

checkpointer = ModelCheckpoint(filepath = "Emotion_weights.hdf5", verbose = 1, save_best_only=True)

history = model.fit(x=x_train, y=y_train)



22/22 [==============================] - 79s 924ms/step - loss: 0.8970 - accuracy: 0.5100
